In [ ]:
import pandas as pd
from urlpath import URL
import holoviews as hv
from holoviews import opts
from bokeh.models import HoverTool
import panel as pn

hv.notebook_extension()
hv.extension('bokeh')

In [ ]:
def get_df(df_path,col_name):
    df = pd.read_csv(df_path)
    melted_df = df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'])
    melted_df.rename(columns={"variable":"Date","value":col_name},inplace=True)
    return melted_df

jhu_repo_url = URL('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/')
path_confirmed = jhu_repo_url / 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
path_deaths = jhu_repo_url / 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
path_recovered = jhu_repo_url / 'csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

df_confirmed = get_df(path_confirmed, 'Confirmed')
df_deaths = get_df(path_deaths, 'Deaths')
df_recovered = get_df(path_recovered, 'Recovered')

df_covid_19 = df_confirmed.join(df_recovered['Recovered']).join(df_deaths['Deaths'])
df_covid_19.rename(columns={'Country/Region':'Country', 'Province/State':'Province'}, inplace=True)

df_covid_19['Date'] = pd.to_datetime(df_covid_19['Date'])
df_covid_19 = df_covid_19.groupby(['Country', 'Date']).agg({'Confirmed':'sum', 'Recovered':'sum', 'Deaths':'sum'}).reset_index()
for col in ['Confirmed', 'Deaths', 'Recovered']:
    df_covid_19[col + '_delta'] = df_covid_19[col].diff()
    df_covid_19.loc[df_covid_19[col + '_delta']<0, col + '_delta'] = 0
    df_covid_19.loc[df_covid_19[col + '_delta'].isna(), col + '_delta'] = 0
# df_covid_19.loc[df_covid_19['Country']=='US'].tail(10)
df_covid_19.describe()

In [ ]:
df = df_covid_19.copy()
df['pat_100'] = df['Confirmed']>=100
df = df.loc[df['pat_100']]
patient_100 = df.groupby('Country')['pat_100'].idxmax()
df_covid_19['days_pat_100'] = None
df['Date_pat_100'] = None
for k, v in patient_100.items():
    df.loc[df.Country == k,'Date_pat_100'] = df.loc[v].Date
df_covid_19['days_pat_100'] = df_covid_19['Date'] - df['Date_pat_100']
df_covid_19['days_pat_100'] = df_covid_19['days_pat_100'].dt.days
# df_covid_19.loc[df_covid_19['days_pat_100'] < 0, 'days_pat_100'] = 0

In [ ]:
top_10_confirmed = df_covid_19.groupby('Country').Confirmed.max().nlargest(10)
countries = list(top_10_confirmed.index)

top_10_deaths = df_covid_19.groupby('Country').Deaths.max().nlargest(10)
top_10_pat_100 = df_covid_19.groupby('Country').days_pat_100.max().nlargest(10)
top_10_conf_delta = df_covid_19.loc[df_covid_19.loc[df_covid_19.Date==df_covid_19.Date.max()].Confirmed_delta.nlargest(10).index][['Country', 'Confirmed_delta']]
top_10_deaths_delta = df_covid_19.loc[df_covid_19.loc[df_covid_19.Date==df_covid_19.Date.max()].Deaths_delta.nlargest(10).index][['Country', 'Deaths_delta']]

t10_conf_plot = hv.Bars(top_10_confirmed).opts(
    invert_axes=True, invert_yaxis=True, xaxis='top', 
    tools=['hover'],
    ylabel='Most Infections (sum)'
)
t10_conf_delta_plot = hv.Bars(top_10_conf_delta).opts(
    invert_axes=True, invert_yaxis=True, xaxis='top', 
    tools=['hover'],
    ylabel='New Infections (last day)',
    color='lightblue'
)
t10_deaths_plot = hv.Bars(top_10_deaths).opts(
    invert_axes=True, invert_yaxis=True, xaxis='top', 
    tools=['hover'], 
    color='indianred',
    ylabel='Deaths (sum)')
t10_pat_100_plot = hv.Bars(top_10_pat_100).opts(
    invert_axes=True, invert_yaxis=True, xaxis='top', 
    tools=[HoverTool(
        tooltips=[
            ('Country', '@Country'),
            ('Days',  "@days_pat_100")
        ])], 
    color='seagreen',
    ylabel='Days since 100 infections'
)
t10_deaths_delta_plot = hv.Bars(top_10_deaths_delta).opts(
    invert_axes=True, invert_yaxis=True, xaxis='top', 
    tools=['hover'],
    ylabel='New Deaths (last day)',
    color='lightcoral'
)
lyt_overview = (t10_conf_plot + t10_conf_delta_plot + t10_pat_100_plot + t10_deaths_plot+t10_deaths_delta_plot).cols(3)
lyt_overview

In [ ]:
plot_width = 640
plot_height= 480
columns = 1

def get_hover(key):
    return HoverTool(tooltips=[
        ('', '@Country'),
        (f'{key[1]}', f'@{key[0]}'),
        ('Date',  "@Date{ %F }")
    ], formatters={"Date": "datetime"})

dataset = hv.Dataset(df_covid_19, kdims=[('Country', 'Country/Region'), 'Date'], 
                vdims=[('Confirmed', 'Confirmed Cases'), ('Confirmed_delta','New Cases'), 'Recovered', 'Deaths', ('Deaths_delta','New Deaths'),])
subset = dataset.select(Country=countries)

confirmed = subset.to(hv.Curve, 'Date', 'Confirmed').opts(
    tools=[get_hover(('Confirmed','Cases'))],
    width=plot_width, height=plot_height,
)
deaths = subset.to(hv.Curve, 'Date', 'Deaths').opts(
    tools=[get_hover(('Deaths', 'Deaths'))],
    width=plot_width, height=plot_height,
)
lyt_cases = (confirmed.overlay().opts(legend_position='top_left') +deaths.overlay().opts(legend_position='top_left')).cols(1)
lyt_cases

In [ ]:
ds_norm = hv.Dataset(df_covid_19, kdims=[('Country', 'Country/Region'), ('days_pat_100', 'Days since Patient 100')], 
                vdims=[('Confirmed', 'Confirmed Cases'), ('Confirmed_delta','New Cases'), 'Recovered', 'Deaths', ('Deaths_delta','New Deaths'),])

def get_hover(key):
    return HoverTool(tooltips=[
        ('', '@Country'),
        (f'{key[1]}', f'@{key[0]}'),
        ('Days',  "@days_pat_100"),
        ('Date',  "@Date{ %F }")
    ], formatters={"Date": "datetime"})

plot_width = 640
plot_height= 480
columns = 1

subset_norm = ds_norm.select(Country=countries, days_pat_100=(0,df_covid_19.loc[df_covid_19['Country']=='Italy']['days_pat_100'].max()+2), Confirmed=(100,df_covid_19['Confirmed'].max()*1.1), Deaths=(1,df_covid_19['Deaths'].max()*1.1))

conf_day_norm = subset_norm.to(hv.Curve, 'days_pat_100', ['Confirmed','Date']).opts(
            tools=[get_hover(('Confirmed','Cases'))],
            logy=True, width=plot_width, height=plot_height,
          )
deaths_day_norm = subset_norm.to(hv.Curve, 'days_pat_100', ['Deaths','Date']).opts(
            tools=[get_hover(('Deaths', 'Deaths'))],
            logy=True, width=plot_width, height=plot_height,
          )
lyt_log = (conf_day_norm.overlay().opts(legend_position='bottom_right') + deaths_day_norm.overlay().opts(legend_position='bottom_right')).cols(columns)
lyt_log

In [ ]:
from bokeh.embed import components
from pathlib import Path

def save_plot(plot, path):
    script, div = components(plot)
    p = Path(path)
    html = script + '\n'.join([f'<h2>{k}</h2>' + v for k,v in div.items()])
    p.write_text(html)
    
save_path = 'docs/_includes/plots.html'
plots = {'Overview': hv.render(lyt_overview), 'Confirmed infections & deaths': hv.render(lyt_cases), 'Logarithmic scale & normalized to days since 100 patients': hv.render(lyt_log)}

save_plot(plots, save_path)

In [ ]:
plot_width = 640
plot_height= 480
columns = 1

def get_hover(key):
    return HoverTool(tooltips=[
        (f'{key[1]}', f'@{key[0]}'),
        ('Date',  "@Date{ %F }")
    ], formatters={"Date": "datetime"})

new_conf = subset.to(hv.Scatter, 'Date', 'Confirmed_delta').opts(
#     color='lightblue', 
    tools=[get_hover(('Confirmed_delta','New'))],
    width=plot_width, height=plot_height,
)
new_deaths = subset.to(hv.Scatter, 'Date', 'Deaths_delta').opts(
    color='indianred', 
    tools=[get_hover(('Deaths_delta','New'))],
    width=plot_width, height=plot_height,
)

plots = confirmed * new_conf + deaths.opts(color='indianred') * new_deaths
lyt_single = pn.panel(plots.cols(columns), center=True, widget_location='top')
lyt_single